In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
import os
import time
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [3]:
np.random.seed(1234)
tf.random.set_seed(1234)

print("TensorFlow version:", tf.__version__)
print("TensorFlow Probability version:", tfp.__version__)

os.makedirs("models", exist_ok=True)

os.makedirs("figures", exist_ok=True)

os.makedirs("data", exist_ok=True)

os.makedirs("streamlit_app", exist_ok=True)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPUs available: {len(gpus)}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs available, using CPU")

TensorFlow version: 2.19.0
TensorFlow Probability version: 0.25.0
No GPUs available, using CPU


In [4]:

#---------------------------------------------------------------------------
# Section 1: Base PINN Network and Loss Functions
#---------------------------------------------------------------------------
class PINN(tf.keras.Model):
    """Base Physics-Informed Neural Network class"""
    
    def __init__(self, hidden_layers, activation='tanh', name="pinn"):
        """
        Initialize PINN model
        
        Parameters:
        -----------
        hidden_layers : list
            List of integers, each specifying the number of neurons in each hidden layer
        activation : str
            Activation function to use in hidden layers
        name : str
            Name of the model
        """
        super(PINN, self).__init__(name=name)
        
        self.hidden_layers = hidden_layers
        self.activation = activation
        
        self.layers_list = []
        
        for units in hidden_layers:
            self.layers_list.append(
                tf.keras.layers.Dense(
                    units, 
                    activation=activation,
                    kernel_initializer=tf.keras.initializers.GlorotNormal()
                )
            )
        
        # For CVD model: outputs are concentration and temp
        self.layers_list.append(
            tf.keras.layers.Dense(
                5,  
                activation=None,
                kernel_initializer=tf.keras.initializers.GlorotNormal()
            )
        )
    
    def call(self, inputs):
        """Forward pass through the network"""
        x = inputs
        for layer in self.layers_list:
            x = layer(x)
        return x
    
    @tf.function
    def get_gradients(self, x, y):
        """
        Compute gradients of output with respect to input
        
        Parameters:
        -----------
        x : tf.Tensor
            Input tensor, shape (batch_size, 3)
            Representing (x, y, t) coordinates
        y : tf.Tensor
            Output tensor, shape (batch_size, 5)
            Representing [SiH4, Si, H2, SiH2, T]
            
        Returns:
        --------
        dict
            Dictionary containing first and second derivatives
        """
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(x)
            with tf.GradientTape(persistent=True) as tape1:
                tape1.watch(x)
                y_pred = self(x)
                
            dy_dx = tape1.batch_jacobian(y_pred, x)
            
            y_x = dy_dx[..., 0]  # derivatives w.r.t. x
            y_y = dy_dx[..., 1]  # derivatives w.r.t. y
            y_t = dy_dx[..., 2]  # derivatives w.r.t. t
            
        # Second derivatives
        dy_xx = tape2.batch_jacobian(y_x, x)[..., 0]  # d²y/dx²
        dy_yy = tape2.batch_jacobian(y_y, x)[..., 1]  # d²y/dy²
        
        del tape1, tape2
        
        # Return all derivatives
        return {
            'dy_dx': dy_dx,
            'y_x': y_x,
            'y_y': y_y,
            'y_t': y_t,
            'y_xx': dy_xx,
            'y_yy': dy_yy
        }

#---------------------------------------------------------------------------
# Section 2: Physical Parameters and Constants for CVD Model
#---------------------------------------------------------------------------

class CVDPhysicalParams:
    """Class to store and manage physical parameters for CVD simulation"""
    
    def __init__(self):
        # Initialize with default parameters for silicon CVD from silane
        
        # Diffusion coefficients (m²/s)
        self.D_SiH4 = 1.0e-5 
        self.D_Si = 5.0e-6    
        self.D_H2 = 4.0e-5    
        self.D_SiH2 = 1.5e-5  
        
        # Thermal properties
        self.thermal_conductivity = 0.1  # W/(m·K)
        self.specific_heat = 700.0      # J/(kg·K)
        self.density = 1.0              # kg/m³
        
        self.A1 = 1.0e6     # Pre-exponential factor
        self.E1 = 1.5e5     # Activation energy (J/mol)
        
        self.A2 = 2.0e5     # Pre-exponential factor
        self.E2 = 1.2e5     # Activation energy (J/mol)

        self.A3 = 3.0e5     # Pre-exponential factor
        self.E3 = 1.0e5     # Activation energy (J/mol)
        
        self.R = 8.314
    
    def get_reaction_rates(self, concentrations, temperature):
        """
        Calculate reaction rates based on Arrhenius equation
        
        Parameters:
        -----------
        concentrations : tf.Tensor
            Concentrations of species [SiH4, Si, H2, SiH2]
        temperature : tf.Tensor
            Temperature
            
        Returns:
        --------
        tuple
            (R1, R2, R3) - Reaction rates for the three reactions
        """
        SiH4, Si, H2, SiH2 = concentrations
        T = temperature
        
        # Reaction 1: SiH4 -> Si + 2H2
        R1 = self.A1 * tf.exp(-self.E1 / (self.R * T)) * SiH4
        
        # Reaction 2: SiH4 + H2 -> SiH2 + 2H2
        R2 = self.A2 * tf.exp(-self.E2 / (self.R * T)) * SiH4 * H2
        
        # Reaction 3: SiH2 + SiH4 -> Si2H6
        R3 = self.A3 * tf.exp(-self.E3 / (self.R * T)) * SiH2 * SiH4
        
        return R1, R2, R3



In [5]:
#---------------------------------------------------------------------------
# Section 3: PDE Residuals for CVD Model
#---------------------------------------------------------------------------

class CVDPDE:
    """Class to compute PDE residuals for CVD simulation"""
    
    def __init__(self, phys_params):
        """
        Initialize with physical parameters
        
        Parameters:
        -----------
        phys_params : CVDPhysicalParams
            Object containing physical parameters
        """
        self.params = phys_params
    
    def compute_residuals(self, x_coords, y_pred, derivatives):
        """
        Compute residuals for all PDEs in the CVD model
        
        Parameters:
        -----------
        x_coords : tf.Tensor
            Input coordinates (x, y, t)
        y_pred : tf.Tensor
            Predicted values [SiH4, Si, H2, SiH2, T]
        derivatives : dict
            Dictionary of derivatives computed by PINN.get_gradients()
            
        Returns:
        --------
        tuple
            (res_SiH4, res_Si, res_H2, res_SiH2, res_T) - Residuals for each equation
        """
        SiH4 = y_pred[:, 0:1]
        Si = y_pred[:, 1:2]
        H2 = y_pred[:, 2:3]
        SiH2 = y_pred[:, 3:4]
        T = y_pred[:, 4:5]
        
        y_t = derivatives['y_t']
        y_xx = derivatives['y_xx']
        y_yy = derivatives['y_yy']
        
        SiH4_t = y_t[:, 0:1]
        Si_t = y_t[:, 1:2]
        H2_t = y_t[:, 2:3]
        SiH2_t = y_t[:, 3:4]
        T_t = y_t[:, 4:5]
        
        # Second spatial derivatives (Laplacian terms)
        SiH4_xx = y_xx[:, 0:1]
        SiH4_yy = y_yy[:, 0:1]
        
        Si_xx = y_xx[:, 1:2]
        Si_yy = y_yy[:, 1:2]
        
        H2_xx = y_xx[:, 2:3]
        H2_yy = y_yy[:, 2:3]
        
        SiH2_xx = y_xx[:, 3:4]
        SiH2_yy = y_yy[:, 3:4]
        
        T_xx = y_xx[:, 4:5]
        T_yy = y_yy[:, 4:5]
        
        # Calculate reaction rates
        R1, R2, R3 = self.params.get_reaction_rates(
            [SiH4, Si, H2, SiH2],
            T
        )
        
        # Residuals for mass transport equations
        # ∂C_i/∂t = D_i ∇²C_i + sum_j(v_ij * R_j)
        
        # Residual for SiH4
        res_SiH4 = SiH4_t - self.params.D_SiH4 * (SiH4_xx + SiH4_yy) + R1 + R2 + R3
        
        # Residual for Si
        res_Si = Si_t - self.params.D_Si * (Si_xx + Si_yy) - R1
        
        # Residual for H2
        res_H2 = H2_t - self.params.D_H2 * (H2_xx + H2_yy) - 2 * R1 - 2 * R2
        
        # Residual for SiH2
        res_SiH2 = SiH2_t - self.params.D_SiH2 * (SiH2_xx + SiH2_yy) - R2 + R3
        
        # Energy equation (heat transfer)
        # ∂T/∂t = κ ∇²T + sum_j(ΔH_j * R_j) / (ρ * Cp)
        
        # Heat source term (simplified)
        Q = 1000 * (R1 + 500 * R2 + 300 * R3)
        
        # Thermal diffusivity
        thermal_diffusivity = self.params.thermal_conductivity / (self.params.density * self.params.specific_heat)
        
        # Residual for temperature
        res_T = T_t - thermal_diffusivity * (T_xx + T_yy) - Q / (self.params.density * self.params.specific_heat)
        
        return res_SiH4, res_Si, res_H2, res_SiH2, res_T



In [6]:
#---------------------------------------------------------------------------
# Section 4: Data Generation for PINN Training
#---------------------------------------------------------------------------

class CVDDataGenerator:
    """Class to generate training data for CVD PINN"""
    
    def __init__(self, domain_bounds):
        """
        Initialize with domain bounds
        
        Parameters:
        -----------
        domain_bounds : dict
            Dictionary with domain bounds (x_min, x_max, y_min, y_max, t_min, t_max)
        """
        self.bounds = domain_bounds
    
    def generate_collocation_points(self, n_points):
        """
        Generate random collocation points for PDE residuals
        
        Parameters:
        -----------
        n_points : int
            Number of points to generate
            
        Returns:
        --------
        np.ndarray
            Array of points, shape (n_points, 3)
        """
        # Generate random points within domain bounds
        x = np.random.uniform(self.bounds['x_min'], self.bounds['x_max'], n_points)
        y = np.random.uniform(self.bounds['y_min'], self.bounds['y_max'], n_points)
        t = np.random.uniform(self.bounds['t_min'], self.bounds['t_max'], n_points)
        
        # Stack coordinates
        collocation_points = np.stack([x, y, t], axis=1)
        
        return collocation_points
    
    def generate_boundary_points(self, n_points_per_boundary):
        """
        Generate boundary points for boundary conditions
        
        Parameters:
        -----------
        n_points_per_boundary : int
            Number of points to generate per boundary
            
        Returns:
        --------
        dict
            Dictionary with boundary points for each boundary
        """
        # Time points - used for all boundaries
        t = np.random.uniform(self.bounds['t_min'], self.bounds['t_max'], n_points_per_boundary)
        
        # Boundary points
        # Lower boundary (y = y_min) - Inlet
        x_lower = np.random.uniform(self.bounds['x_min'], self.bounds['x_max'], n_points_per_boundary)
        y_lower = np.ones_like(x_lower) * self.bounds['y_min']
        
        # Upper boundary (y = y_max) - Substrate
        x_upper = np.random.uniform(self.bounds['x_min'], self.bounds['x_max'], n_points_per_boundary)
        y_upper = np.ones_like(x_upper) * self.bounds['y_max']
        
        # Left boundary (x = x_min) - Wall
        y_left = np.random.uniform(self.bounds['y_min'], self.bounds['y_max'], n_points_per_boundary)
        x_left = np.ones_like(y_left) * self.bounds['x_min']
        
        # Right boundary (x = x_max) - Wall
        y_right = np.random.uniform(self.bounds['y_min'], self.bounds['y_max'], n_points_per_boundary)
        x_right = np.ones_like(y_right) * self.bounds['x_max']
        
        # Stack coordinates
        boundary_points = {
            'inlet': np.stack([x_lower, y_lower, t], axis=1),
            'substrate': np.stack([x_upper, y_upper, t], axis=1),
            'left_wall': np.stack([x_left, y_left, t], axis=1),
            'right_wall': np.stack([x_right, y_right, t], axis=1)
        }
        
        return boundary_points
    
    def generate_initial_points(self, n_points):
        """
        Generate initial condition points at t = t_min
        
        Parameters:
        -----------
        n_points : int
            Number of points to generate
            
        Returns:
        --------
        np.ndarray
            Array of points, shape (n_points, 3)
        """
        # Generate random spatial points
        x = np.random.uniform(self.bounds['x_min'], self.bounds['x_max'], n_points)
        y = np.random.uniform(self.bounds['y_min'], self.bounds['y_max'], n_points)
        t = np.ones_like(x) * self.bounds['t_min']
        
        # Stack coordinates
        initial_points = np.stack([x, y, t], axis=1)
        
        return initial_points

    def generate_uniform_grid(self, nx, ny, nt):
        """
        Generate uniform grid for visualization
        
        Parameters:
        -----------
        nx : int
            Number of points in x direction
        ny : int
            Number of points in y direction
        nt : int
            Number of points in t direction
            
        Returns:
        --------
        np.ndarray
            Array of grid points, shape (nx*ny*nt, 3)
        """
        # Generate grid points
        x = np.linspace(self.bounds['x_min'], self.bounds['x_max'], nx)
        y = np.linspace(self.bounds['y_min'], self.bounds['y_max'], ny)
        t = np.linspace(self.bounds['t_min'], self.bounds['t_max'], nt)
        
        # Create meshgrid
        X, Y, T = np.meshgrid(x, y, t, indexing='ij')
        
        # Stack coordinates
        grid_points = np.stack([X.flatten(), Y.flatten(), T.flatten()], axis=1)
        
        return grid_points, (nx, ny, nt)



In [7]:
#---------------------------------------------------------------------------
# Section 5: Entropy-Langevin Dynamics for PINN Training
#---------------------------------------------------------------------------

class EntropyRegularizedLoss:
    """Class to compute entropy-regularized loss for ensemble of PINNs"""
    
    def __init__(self, alpha=0.1, beta=10.0):
        """
        Initialize with entropy and temperature parameters
        
        Parameters:
        -----------
        alpha : float
            Entropy weight parameter
        beta : float
            Inverse temperature parameter
        """
        self.alpha = tf.Variable(alpha, trainable=False, dtype=tf.float32)
        self.beta = tf.Variable(beta, trainable=False, dtype=tf.float32)
    
    def compute_loss(self, losses, current_loss, current_gradient):
        """
        Compute entropy-regularized loss and modified gradient
        
        Parameters:
        -----------
        losses : list
            List of loss values for each network in the ensemble
        current_loss : tf.Tensor
            Loss value for the current network
        current_gradient : tf.Tensor
            Gradient of the loss for the current network
            
        Returns:
        --------
        tuple
            (modified_loss, modified_gradient)
        """
        # Convert losses to tensor
        losses_tensor = tf.convert_to_tensor(losses, dtype=tf.float32)
        
        # Compute mean loss and mean gradient across ensemble
        mean_loss = tf.reduce_mean(losses_tensor)
        
        # Compute entropy term
        # S[p(θ)] ≈ log(mean(exp(-β*L(θ))))/β + constant
        entropy_term = tf.math.log(tf.reduce_mean(tf.exp(-self.beta * losses_tensor))) / self.beta
        
        # Compute entropy-regularized loss
        # L_tilde(θ) = L(θ) - α * S[p(θ)]
        modified_loss = current_loss - self.alpha * entropy_term
        
        # Compute modified gradient
        # ∇_θ L_tilde(θ) = ∇_θ L(θ) - α*β*(∇_θ L(θ) - mean(∇_θ L(θ)))
        gradient_difference = current_gradient - mean_loss
        modified_gradient = current_gradient - self.alpha * self.beta * gradient_difference
        
        return modified_loss, modified_gradient
    
    def update_parameters(self, iteration, max_iterations):
        """
        Update alpha and beta parameters based on scheduling
        
        Parameters:
        -----------
        iteration : int
            Current iteration
        max_iterations : int
            Maximum number of iterations
        """
        # Implement scheduling as described in the paper
        # This is a simple linear scheduling for demonstration
        progress = tf.cast(iteration / max_iterations, tf.float32)
        
        # Decrease alpha over time (less entropy regularization)
        alpha_new = 0.1 * (1.0 - 0.9 * progress)
        
        # Increase beta over time (lower temperature, less noise)
        beta_new = 10.0 * (1.0 + 9.0 * progress)
        
        # Update parameters
        self.alpha.assign(alpha_new)
        self.beta.assign(beta_new)



In [8]:
#---------------------------------------------------------------------------
# Section 6: Testing Framework and Visualization
#---------------------------------------------------------------------------

# Initialize physical parameters
cvd_params = CVDPhysicalParams()
print("\nPhysical parameters initialized:")
print(f"Diffusion coefficients: D_SiH4 = {cvd_params.D_SiH4}, D_Si = {cvd_params.D_Si}, D_H2 = {cvd_params.D_H2}, D_SiH2 = {cvd_params.D_SiH2}")
print(f"Thermal parameters: k = {cvd_params.thermal_conductivity}, Cp = {cvd_params.specific_heat}, ρ = {cvd_params.density}")
print(f"Reaction parameters: A1 = {cvd_params.A1}, E1 = {cvd_params.E1}, A2 = {cvd_params.A2}, E2 = {cvd_params.E2}, A3 = {cvd_params.A3}, E3 = {cvd_params.E3}")
print(f"Gas constant: R = {cvd_params.R}")

# Define domain bounds
domain_bounds = {
    'x_min': 0.0,
    'x_max': 0.1,
    'y_min': 0.0,
    'y_max': 0.05,
    't_min': 0.0,
    't_max': 10.0
}

print("\nDomain bounds:")
for key, value in domain_bounds.items():
    print(f"{key} = {value}")

# Create a simple PINN model to test
test_model = PINN([20, 20, 20], activation='tanh', name="test_model")

# Create a test input
test_input = tf.random.uniform((5, 3))
print("\nTest input shape:", test_input.shape)

# Get test output
test_output = test_model(test_input)
print("Test output shape:", test_output.shape)

# Compute test gradients
test_grads = test_model.get_gradients(test_input, test_output)
print("\nComputed gradients:")
for key, value in test_grads.items():
    print(f"{key} shape: {value.shape}")

# Create PDE residual calculator
cvd_pde = CVDPDE(cvd_params)

# Compute test residuals
test_residuals = cvd_pde.compute_residuals(test_input, test_output, test_grads)
print("\nComputed residuals:")
for i, res in enumerate(test_residuals):
    print(f"Residual {i+1} shape: {res.shape}")

# Create data generator and generate sample data
data_gen = CVDDataGenerator(domain_bounds)

# Test data generation
test_collocation = data_gen.generate_collocation_points(10)
print("\nGenerated collocation points shape:", test_collocation.shape)

test_boundary = data_gen.generate_boundary_points(5)
print("Generated boundary points:")
for key, value in test_boundary.items():
    print(f"{key} shape: {value.shape}")

test_initial = data_gen.generate_initial_points(10)
print("Generated initial points shape:", test_initial.shape)

# Test entropy-regularized loss
entropy_loss = EntropyRegularizedLoss(alpha=0.1, beta=10.0)
test_losses = [1.0, 1.2, 0.9, 1.1, 1.0]
test_current_loss = tf.constant(1.1, dtype=tf.float32)
test_current_gradient = tf.constant(2.0, dtype=tf.float32)

modified_loss, modified_gradient = entropy_loss.compute_loss(
    test_losses, test_current_loss, test_current_gradient
)

print("\nEntropy regularization test:")
print(f"Original loss: {test_current_loss.numpy()}")
print(f"Modified loss: {modified_loss.numpy()}")
print(f"Original gradient: {test_current_gradient.numpy()}")
print(f"Modified gradient: {modified_gradient.numpy()}")

# Test parameter scheduling
entropy_loss.update_parameters(0, 1000)
print(f"Updated alpha: {entropy_loss.alpha.numpy()}")
print(f"Updated beta: {entropy_loss.beta.numpy()}")

entropy_loss.update_parameters(500, 1000)
print(f"Updated alpha at halfway: {entropy_loss.alpha.numpy()}")
print(f"Updated beta at halfway: {entropy_loss.beta.numpy()}")

entropy_loss.update_parameters(1000, 1000)
print(f"Updated alpha at end: {entropy_loss.alpha.numpy()}")
print(f"Updated beta at end: {entropy_loss.beta.numpy()}")

print("\nNotebook 1 completed successfully!")


Physical parameters initialized:
Diffusion coefficients: D_SiH4 = 1e-05, D_Si = 5e-06, D_H2 = 4e-05, D_SiH2 = 1.5e-05
Thermal parameters: k = 0.1, Cp = 700.0, ρ = 1.0
Reaction parameters: A1 = 1000000.0, E1 = 150000.0, A2 = 200000.0, E2 = 120000.0, A3 = 300000.0, E3 = 100000.0
Gas constant: R = 8.314

Domain bounds:
x_min = 0.0
x_max = 0.1
y_min = 0.0
y_max = 0.05
t_min = 0.0
t_max = 10.0

Test input shape: (5, 3)
Test output shape: (5, 5)

Computed gradients:
dy_dx shape: (5, 5, 3)
y_x shape: (5, 5)
y_y shape: (5, 5)
y_t shape: (5, 5)
y_xx shape: (5, 5)
y_yy shape: (5, 5)

Computed residuals:
Residual 1 shape: (5, 1)
Residual 2 shape: (5, 1)
Residual 3 shape: (5, 1)
Residual 4 shape: (5, 1)
Residual 5 shape: (5, 1)

Generated collocation points shape: (10, 3)
Generated boundary points:
inlet shape: (5, 3)
substrate shape: (5, 3)
left_wall shape: (5, 3)
right_wall shape: (5, 3)
Generated initial points shape: (10, 3)

Entropy regularization test:
Original loss: 1.100000023841858
Modif